In [ ]:
###################Encryption, Update and Keystream function definition Start#####################

def keystream(S,B):                                       #Keystream bit generation function
    L = B[7] ^ B[11] ^ B[30] ^ B[40] ^ B[45] ^ B[54] ^ B[71]
    Q = B[4]&B[21] ^ B[9]&B[52] ^ B[18]&B[37] ^ B[44]&B[76]
    T = B[5] ^ B[8]&B[82] ^ B[34]&B[67]&B[73] ^ B[2]&B[28]&B[41]&B[65] ^ B[13]&B[29]&B[50]&B[64]&B[75] \
    ^ B[6]&B[14]&B[26]&B[32]&B[47]&B[61] ^ B[1]&B[19]&B[27]&B[43]&B[57]&B[66]&B[78]
    T_c = S[23] ^ S[3]&S[16] ^ S[9]&S[13]&B[48] ^ S[1]&S[24]&B[38]&B[63]
    
    out = L ^ Q ^ T ^ T_c
    return out


def NFSR1_bk_update(S):                                   #Feedback function for NFSR1 in backward direction
    temp = S[30]  ^ S[1]  ^  S[4]  ^ S[5]  ^  S[14] ^ S[16] ^ S[17] ^ S[19] ^  S[24] ^ S[7]&S[17] ^ S[7]&S[19] ^ S[11]&S[20] \
        ^ S[13]&S[18] ^ S[16]&S[20] ^ S[19]&S[21] ^ S[3]&S[11]&S[21] ^ S[3]&S[18]&S[21] ^ S[6]&S[19]&S[20] \
        ^ S[7]&S[17]&S[21] ^ S[7]&S[19]&S[21] ^ S[11]&S[18]&S[21] ^ S[19]&S[20]&S[21] ^ S[3]&S[6]&S[11]&S[20]\
        ^ S[3]&S[6]&S[18]&S[20] ^ S[3]&S[11]&S[20]&S[21] ^ S[3]&S[18]&S[20]&S[21] ^ S[6]&S[7]&S[17]&S[20] \
        ^ S[6]&S[7]&S[19]&S[20] ^ S[6]&S[11]&S[18]&S[20] ^ S[7]&S[17]&S[20]&S[21] ^ S[7]&S[19]&S[20]&S[21]\
        ^ S[11]&S[18]&S[20]&S[21]
    
    return temp


def NFSR2_bk_update(B,s0):                                #Feedback function for NFSR2 in backward direction
    temp = s0 ^ B[89] ^ B[23] ^ B[48] ^ B[78] ^ B[83] ^ B[2]&B[58] ^ B[9]&B[11] ^ B[14]&B[15] ^ B[24]&B[52] \
    ^ B[34]&B[41] ^ B[54]&B[57] ^ B[59]&B[73] ^ B[19]&B[21]&B[22] ^ B[61]&B[67]&B[71] \
    ^ B[76]&B[79]&B[80]&B[82]
    
    return temp
    
def NFSR1_fw_update(S):                                   #Feedback function for NFSR1 in forward direction
    temp = S[0]  ^ S[2]  ^  S[5]  ^ S[6]  ^  S[15] ^ S[17] ^ S[18] ^ S[20] ^  S[25] ^ S[8]&S[18] ^ S[8]&S[20] ^ S[12]&S[21] \
        ^ S[14]&S[19] ^ S[17]&S[21] ^ S[20]&S[22] ^ S[4]&S[12]&S[22] ^ S[4]&S[19]&S[22] ^ S[7]&S[20]&S[21] \
        ^ S[8]&S[18]&S[22] ^ S[8]&S[20]&S[22] ^ S[12]&S[19]&S[22] ^ S[20]&S[21]&S[22] ^ S[4]&S[7]&S[12]&S[21]\
        ^ S[4]&S[7]&S[19]&S[21] ^ S[4]&S[12]&S[21]&S[22] ^ S[4]&S[19]&S[21]&S[22] ^ S[7]&S[8]&S[18]&S[21] \
        ^ S[7]&S[8]&S[20]&S[21] ^ S[7]&S[12]&S[19]&S[21] ^ S[8]&S[18]&S[21]&S[22] ^ S[8]&S[20]&S[21]&S[22]\
        ^ S[12]&S[19]&S[21]&S[22]
    
    return temp

    
def NFSR2_fw_update(B,s0):                                #Feedback function for NFSR2 in forward direction
    temp = s0 ^ B[0] ^ B[24] ^ B[49] ^ B[79] ^ B[84] ^ B[3]&B[59] ^ B[10]&B[12] ^ B[15]&B[16] ^ B[25]&B[53] \
    ^ B[35]&B[42] ^ B[55]&B[58] ^ B[60]&B[74] ^ B[20]&B[22]&B[23] ^ B[62]&B[68]&B[72] \
    ^ B[77]&B[80]&B[81]&B[83]
    
    return temp


def Encryption():                                          #Encryption Function
    #Initialisation Phase I
    import random
    K = [random.randint(0,1) for i in range(120)]          #Secret Key
    IV = [random.randint(0,1) for i in range(64)]          #Initialisation Vector

    S = [0]*31                                          #NFSR 1
    B = [0]*90                                          #NFSR 2

    #Phase 1: Key and IV Loading
    for i in range(90):
        if i<=63:
            B[i] = (K[i] ^ IV[i])
        else:
            B[i] = K[i]

    for i in range(29):
        S[i] = K[i + 90]
        
    S[29] = (K[119] ^ 1)
    S[30] = 1

    #Phase 2: Grain-like Mixing
    for i in range(128):
        z = keystream(S,B)
        b = NFSR2_fw_update(B,S[0])
        s = NFSR1_fw_update(S)
        
        for k in range(89):
            B[k] = B[k+1]
        B[89] = b^z

        for k in range(30):
            S[k] = S[k+1]
        S[30] = s^z

    #Phase 3: Second Key Addition
    for i in range(90):
        B[i] = (B[i]^K[i])
    for i in range(30):
        S[i] = (S[i]^K[90+i])
    S[30] = 1
    
    Z_c = [0]*128

    #Phase 4: Final Diffusion
    for i in range(128):
        Z_c[i] = keystream(S,B)
        b = NFSR2_fw_update(B,S[0])
        s = NFSR1_fw_update(S)

        for k in range(89):
            B[k] = B[k+1]
        B[89] = b

        for k in range(30):
            S[k] = S[k+1]
        S[30] = s

    #Pseudorandom phase
    N_round = 500                #Targeted round in psuedorandom phase (any random number)
    for i in range(N_round):
       # z = keystream(S,B)
        b = NFSR2_fw_update(B,S[0])
        s = NFSR1_bk_update(S)

        for k in range(89):
            B[k] = B[k+1]
        B[89] = b

        for k in range(30):
            S[k] = S[k+1]
        S[30] = s
        
    return S,B
####################Encryption, Update and Keystream function definition End#####################



#Lizard Encryption
#Specificaton
#121-bit internal state
#NFSR1 31-bit  NFSR2 90-bit
#NFSR1 S       NFSR2 B
#Key 120-bit
#IV 64-bit

import math
import random
import time
from datetime import datetime

state_len = 121      #state_size
r = [31, 90]         #Size of the registers NFSR 1 (S) and NFSR 2 (B)

mc_len = 32         #microcontroller length
N_round = 100       #Total number of rounds needed
G_pos_S = []        #Guessing position NFSR1
G_pos_B = []        #Guessing position NFSR
sample = 1
z_act = 1
tolerance = 1
Error = [i for i in range(-tolerance,tolerance+1,1)]


guess = len(G_pos_S)+len(G_pos_B)    #Number of guessed bits

Nblock = math.ceil(state_len/mc_len)      #No. of Blocks

bitlen = math.floor(math.log(mc_len,2)) + 1   #Number of bit that is needed to represent the max. HW of any blocks
    


N_round_f = math.ceil(N_round/2)                #No. of forward rounds
N_round_b = math.floor(N_round/2)               #No. of backward rounds

print("Date and Time ",datetime.now())
print("\nHW/",mc_len," Model Lizard")
print("Number of rounds in forward ", N_round_f," and in backward ", N_round_b)
print("Total number of rounds",N_round)
print("The Guessed bits are NFSR1 ", G_pos_S,"\nNFSR2 ",G_pos_B)
print("Number of Guessed bit ", guess)
print("Error Tolerance class ", Error)
print("Keystream Included", z_act)
print("Microcontroller Used ", mc_len)
print("Sample used", sample)
print("Date and Time ",datetime.now())

filename = 'Lizard_HW-'+str(mc_len)+'_g'+str(guess)+'_z_'+str(z_act)+'_tolerance-'+\
str(tolerance)+'_fwd+bkd_Rounds'+str(N_round)+'.txt'

file = open(filename,'a')
file.write("\n\n\nDate and Time " + str(datetime.now()))
file.write("\nHW/"+str(mc_len)+" Model Lizard")
file.write("\nNumber of rounds in forward " + str(N_round_f) +" and in backward " + str(N_round_b))
file.write("\nTotal number of rounds" + str(N_round))
file.write("\nThe Guessed bits are NFSR1 " + str(G_pos_S) + "\nNFSR2 " + str(G_pos_B))
file.write("\nNumber of Guessed bit "+ str(guess))
file.write("\nError Tolerance class "+ str(Error))
file.close()

cnt_success = 0
runtime = [0]*sample

for loop in range(sample):
    ###############Lizard cipher with a random key and IV to generate keystream and HW ####################################
    
    S_org, B_org = Encryption()         #Internal state picked from the pseudorandom phase


    S = S_org[:]
    B = B_org[:]

    Z_f = [0]*N_round_f              #Array to store the forward keystream bits
    Z_b = [0]*N_round_b              #Array to store the backward keystream bits

    #Array to store the HW of each consecutive blocks in forward directions
    hamm_wt_f = [[0 for i in range(Nblock)] for j in range(N_round_f)]   

    #Array to store the HW of each consecutive blocks in backward directions
    hamm_wt_b = [[0 for i in range(Nblock)] for j in range(N_round_b)]


    #Information on. keystream and HW/32 in backward direction
    for i in range(N_round_b): 
        s = NFSR1_bk_update(S)            #Feedback function of NFSR 1
        b = NFSR2_bk_update(B,s)          #Feedback function of NFSR 2

        S = [s] + S[:r[0]-1]                  #State update
        B = [b] + B[:r[1]-1] 

        temp = B + S

        Z_b[i] = keystream(S,B)            #Storing the keystream bits at each round in backward direction


        #Storing the HW of each block for internal  states in backward direction
        count = 0
        for l in range(Nblock-1):
            for k in range(mc_len):
                hamm_wt_b[i][l] += temp[count]                
                count += 1

        while(count < state_len):                     #Helpful when state_len is not a multiple of mc_len
            hamm_wt_b[i][-1] += temp[count]
            count += 1



    S = S_org[:]
    B = B_org[:]
    #Information on keystream and HW/32 in forward direction
    for i in range(N_round_f):
        Z_f[i] = keystream(S,B)            #Storing the keystream bits at each round in forward direction

        temp = B + S
        count = 0

        #Storing the actual HW of each block for internal  states in forward direction
        for l in range(Nblock-1):
            for k in range(mc_len):
                hamm_wt_f[i][l] += temp[count]
                count += 1

        while(count < state_len):            #Helpful when state_len is not a multiple of mc_len
            hamm_wt_f[i][-1] += temp[count]
            count += 1

        if i < N_round_f -1:
            #updating the registers
            s = NFSR1_fw_update(S)
            b = NFSR2_fw_update(B,S[0])

            S = S[1:r[0]] + [s]
            B = B[1:r[1]] + [b]
    
    #############Injection Error to the Original Hamming Weight ##########################
    cntf = 0
    cntb = 0
    if len(Error) > 0:
        for i in range(N_round_b):
            for j in range(Nblock):
                temp = random.choice(Error)
                if temp != 0:
                    cntb += 1
                hamm_wt_b[i][j] += temp
                if hamm_wt_b[i][j] < 0:
                    hamm_wt_b[i][j] = 0
                if hamm_wt_b[i][j] > mc_len:
                    hamm_wt_b[i][j] = mc_len

        for i in range(N_round_f):
            for j in range(Nblock):
                temp = random.choice(Error)
                if temp != 0:
                    cntf += 1
                hamm_wt_f[i][j] += temp
                if hamm_wt_f[i][j] < 0:
                    hamm_wt_f[i][j] = 0
                if hamm_wt_f[i][j] > mc_len:
                    hamm_wt_f[i][j] = mc_len
            
    

    ###############SMT Modelling to recover state bits from keystream and HW/32#################
    from z3 import *
    m = Solver()

    #Defining variables of the format BitVec(.,1)
    S_var_org = [BitVec('s%d'%i,1) for i in range(r[0])]          #For register S
    B_var_org = [BitVec('b%d'%i,1) for i in range(r[1])]          #For register B

    S_f = [BitVec('S_f%d'%i,1) for i in range(1,N_round_f,1)]       #Dummy variables for register S in forward direction
    B_f = [BitVec('B_f%d'%i,1) for i in range(1,N_round_f,1)]       #Dummy variables for register B in forward direction


    S_b = [BitVec('S_b%d'%i,1) for i in range(1,N_round_b+1,1)]     #Dummy variables for register S in backward direction
    B_b = [BitVec('B_b%d'%i,1) for i in range(1,N_round_b+1,1)]     #Dummy variables for register B in backward direction




    S = S_var_org[:]
    B = B_var_org[:]


    #To check the satisfiability of equations guess all variables otherwise guess as per the requirement
    #L_org, N_org are the original register of the targeted internal state
    #Guess accordingly 
    for i in G_pos_S:
        m.add(S[i] == S_org[i])
    for i in G_pos_B:
        m.add(B[i] == B_org[i])
    #Guess End


    #Array to store the keystream equations    
    Z_f_equ = [0]*N_round_f        
    Z_b_equ = [0]*N_round_b

    #Array to store the dummy variable equations for both register S and B
    S_f_equ = [0]*(N_round_f-1)
    B_f_equ = [0]*(N_round_f-1)

    S_b_equ = [0]*N_round_b
    B_b_equ = [0]*N_round_b

    #Array to store the HW/32 equations
    hamm_wt_f_equ = [[0 for i in range(Nblock)] for i in range(N_round_f)] 
    hamm_wt_b_equ = [[0 for i in range(Nblock)] for i in range(N_round_b)] 


    #Equation generation steps for backward internal states
    for i in range(N_round_b):

        #Dummy Variable equated to the feedback function
        s = NFSR1_bk_update(S)
        b = NFSR2_bk_update(B,s)

        S_b_equ[i] = (S_b[i] == s)          
        B_b_equ[i] = (B_b[i] == b)

        #State updated with dummy variables
        S = [S_b[i]] + S[:r[0]-1]
        B = [B_b[i]] + B[:r[1]-1]

        #Hamming weight equation
        temp = B + S
        count = 0
        for l in range(Nblock-1):
            for k in range(mc_len):
                hamm_wt_b_equ[i][l] += ZeroExt(bitlen-1,temp[count])
                count+=1
            m.add(ULE(hamm_wt_b_equ[i][l],min(hamm_wt_b[i][l]+tolerance,mc_len)))
            m.add(UGE(hamm_wt_b_equ[i][l],max(hamm_wt_b[i][l]-tolerance,0)))
            
        while(count < state_len):                               #Helpful when state_len is not a multiple of mc_len
            hamm_wt_b_equ[i][-1] += ZeroExt(bitlen-1,temp[count])
            count += 1
        m.add(ULE(hamm_wt_b_equ[i][-1],min(hamm_wt_b[i][-1]+tolerance,mc_len)))
        m.add(UGE(hamm_wt_b_equ[i][-1],max(hamm_wt_b[i][-1]-tolerance,0)))

        #Keystream equation
        Z_b_equ[i] = (keystream(S,B) == Z_b[i])


    S = S_var_org[:]
    B = B_var_org[:]
    #Equations generation steps for forward internal states
    for i in range(N_round_f):
        Z_f_equ[i] = (keystream(S,B) == Z_f[i])             #Keystream equation

        #Hamming weight equation
        temp = B + S
        count = 0
        for l in range(Nblock-1):
            for k in range(mc_len):
                hamm_wt_f_equ[i][l] += ZeroExt(bitlen-1,temp[count])
                count+=1
            m.add(ULE(hamm_wt_f_equ[i][l],min(hamm_wt_f[i][l]+tolerance,mc_len)))
            m.add(UGE(hamm_wt_f_equ[i][l],max(hamm_wt_f[i][l]-tolerance,0)))

        while(count< state_len):                            #Helpful when state_len is not a multiple of mc_len
            hamm_wt_f_equ[i][-1] += ZeroExt(bitlen-1,temp[count])
            count += 1
        m.add(ULE(hamm_wt_f_equ[i][-1],min(hamm_wt_f[i][-1]+tolerance,mc_len)))
        m.add(UGE(hamm_wt_f_equ[i][-1],max(hamm_wt_f[i][-1]-tolerance,0)))

        if i < N_round_f-1:
            s = NFSR1_fw_update(S)
            b = NFSR2_fw_update(B,S[0])

            S_f_equ[i] = (S_f[i] == s)            #Dummy variable equated to the feedback function
            B_f_equ[i] = (B_f[i] == b)

            #Internal state updated with dummy variable
            S = S[1:] + [S_f[i]]
            B = B[1:] + [B_f[i]]


    # All Keystream and Dummy variable equations
    if z_act == 1:
        Equ = Z_f_equ + Z_b_equ + S_f_equ + B_f_equ + S_b_equ + B_b_equ
    else:
        Equ = S_f_equ + B_f_equ + S_b_equ + B_b_equ


    m.add(Equ)              #Equation feeded to the model m
    start = time.time()
    temp = m.check()        #Check the satisfiability of the given system of equations
    end = time.time()
    
    file = open(filename,'a')
    #if Satisfiable
    if temp == sat:         
        print(temp)
        soln = m.model()           #soln contains the output
        print("SMT Time = ", end - start)         #Time taken by solver to solve
        file.write("\nSMT Time = "+str(end-start))
        runtime[loop] = end-start
        #Extracting the value of the internal state variables
        out_S = [0]*(r[0])
        out_B = [0]*(r[1])
        for i in range(r[0]):
            out_S[i] = soln[S_var_org[i]]
        for i in range(r[1]):
            out_B[i] = soln[B_var_org[i]]

        #Check whether the original state is same as the output state
        #S_org, B_org are the original registers whereas out_S and out_B are the output of SMT
        if S_org == out_S and B_org == out_B:
            print("***************************matched**********************************")
            file.write("\n***************************matched**********************************")
            cnt_success += 1
        else:
            print("############################not matched##############################################")
            file.write("\n###################################not matched###################################################")
            print(S_org)
            print(B_org)
            print(out_S)
            print(out_B)

    #If the solution is Unsatisfiable, check code and equations
    else:
        print("###################################unsat###################################################")
        file.write("\n############################unsat##################################################")
        print(temp)
        print(S_org)
        print(B_org)
    file.close()
    
    
print("No. of Sucessful output ",cnt_success,"  out of ",sample)
file = open(filename,'a')
file.write("\n\n No. of correct ouptput ")
file.write(str(cnt_success))
file.write(" out of sample ")
file.write(str(sample))
file.write("\n")
file.write("Runtime = ")
file.write(str(runtime))
import numpy
runtime = numpy.array(runtime)
file.write("\n\n Mean =  ")
file.write(str(runtime.mean()))
file.write("\n Std = ")
file.write(str(runtime.std()))
file.write("\n")
file.close()


In [ ]:
print(cntf,cntb)

In [ ]:
print(len(hamm_wt_f[0]))